In [43]:
import glob
import itertools
import json
import os
import pandas

pandas.options.display.max_rows = None

In [44]:
def parseFileNumber(fileName):
    fragments = fileName.split('_')
    return int(fragments[2])

In [45]:
# path = "ExperimentData//"
# path = "SampleSessionLayerableOnly//"
# path = "1652473795203//"
# path = "1652484257598//"
# path = "31/PersonalAR_2//"

session_paths = glob.glob("DataCollection-20220517T162747Z-001/DataCollection/85/PersonalAR/*")
print(session_paths)
dir_list = []
for session_path in session_paths:
    dir_list.extend(os.listdir(session_path))

dir_list.sort(key=parseFileNumber)

print("Using session paths: ")
print(session_paths)
# print("Files and directories in '", path, "' :")
 
# prints all files
# print ("Found following session files: ")
# print(dir_list)
# print(dir_list)


['DataCollection-20220517T162747Z-001/DataCollection/85/PersonalAR\\1653694769600', 'DataCollection-20220517T162747Z-001/DataCollection/85/PersonalAR\\1653696859445', 'DataCollection-20220517T162747Z-001/DataCollection/85/PersonalAR\\1653697339969', 'DataCollection-20220517T162747Z-001/DataCollection/85/PersonalAR\\1653697449084']
Using session paths: 
['DataCollection-20220517T162747Z-001/DataCollection/85/PersonalAR\\1653694769600', 'DataCollection-20220517T162747Z-001/DataCollection/85/PersonalAR\\1653696859445', 'DataCollection-20220517T162747Z-001/DataCollection/85/PersonalAR\\1653697339969', 'DataCollection-20220517T162747Z-001/DataCollection/85/PersonalAR\\1653697449084']


In [46]:
appEvents = []
gestureEvents = []
codesEvents = []
experimentEvents = []

def parseData(json_data):
    frames_list = []
    frames = json_data["frames"]
    for frame in frames:
        frameData = {
            "frameNum": int,
            "timestamp": float,
            "systemTime": str,
            "unixTime": float,
            "gazeValid": bool,

            "hPos": {"x":float,"y":float,"z":float},
            "hDir": {"x":float,"y":float,"z":float},
            "hRot": {"i":float,"j":float,"k":float},
            "hAngl": {"x":float,"y":float,"z":float},

            "gazeOrigin": {"x":float,"y":float,"z":float},
            "gazeDirection": {"x":float,"y":float,"z":float},

            "hand":{},
            "rightHandRay": {"x":float,"y":float,"z":float},
            "leftHandRay": {"x":float,"y":float,"z":float},
            "openApps": [],
            "appEvents": [],
            "placedObjects": [],
            "anchorEvents":[],
            "codesEvents":[],
            "gestureEvents":[],
            "IsLayerableMode": bool,
            "experimentEvents": []
        }

        appEvent = frame["appEvents"]
        codeEvent = frame["codesEvents"]
        experimentEvent = frame["experimentEvents"]
        gestureEvent = frame["gestureEvents"]

        frameData["frameNum"] = frame["frameNum"]
        frameData["timestamp"] = frame["timestamp"]
        frameData["systemTime"] = frame["systemTime"]
        frameData["unixTime"] = frame["unixTime"]
        frameData["gazeValid"] = frame["gazeValid"]
        frameData["hPos"] = frame["hPos"]
        frameData["hDir"] = frame["hDir"]
        frameData["hRot"] = frame["hRot"]
        frameData["hAngl"] = frame["hAngl"]

        frameData["gazeOrigin"] = frame["gazeOrigin"]
        frameData["gazeDirection"] = frame["gazeDirection"]

        frameData["hand"] = frame["hand"]
        frameData["rightHandRay"] = frame["rightHandRay"]
        frameData["leftHandRay"] = frame["leftHandRay"]
        frameData["openApps"] = frame["openApps"]
        frameData["appEvents"] = appEvent
        frameData["placedObjects"] = frame["placedObjects"]
        frameData["anchorEvents"] = frame["anchorEvents"]
        frameData["codesEvents"] = codeEvent
        frameData["gestureEvents"] = gestureEvent
        frameData["IsLayerableMode"] = frame["IsLayerableMode"]
        frameData["experimentEvents"] = experimentEvent
        frames_list.append(frameData)
        if appEvent:
            appEvents.append(appEvent)
        if codeEvent:
            codesEvents.append(codeEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)
        if gestureEvent:
            gestureEvents.append(gestureEvent)
    return frames_list


In [47]:
frames = []

def extract_frames(session_path):
    session_dir_list = os.listdir(session_path)
    extracted_frames = []
    for filename in session_dir_list:
        if filename.endswith(".json"):
            # Prints only text file present in My Folder
            #print(filename)
            firstFileFlag = False
            if "Session_0" in filename:
                firstFileFlag = True
            with open(os.path.join(session_path, filename), 'r') as f:
                json_data = json.load(f)
                if(firstFileFlag):
                    #data.append(json_data["obj"]["sessionRecording"])
                    session_frames = parseData(json_data["obj"]["sessionRecording"])
                    extracted_frames.append(session_frames)
                else:
                    #data.append(json.load(f))
                    session_frames = parseData(json_data)
                    extracted_frames.append(session_frames)
    return extracted_frames

for session_path in session_paths:
    frames.extend(extract_frames(session_path))

In [48]:
frames
framesUnpacked = list(itertools.chain(*frames))
framesDf = pandas.DataFrame(framesUnpacked)

framesDf = framesDf.sort_values(by='timestamp')
framesDf.head(1)
framesDf.head(2)

,frameNum,timestamp,systemTime,unixTime,gazeValid,hPos,hDir,hRot,hAngl,gazeOrigin,...,rightHandRay,leftHandRay,openApps,appEvents,placedObjects,anchorEvents,codesEvents,gestureEvents,IsLayerableMode,experimentEvents
8539,1,2300,17-14-21-74,1653696861745,False,"{'x': 0.0066076358780264854, 'y': 0.0001536311...","{'x': 0.0981193482875824, 'y': -0.230901017785...","{'x': 0.11576935648918152, 'y': 0.050880745053...","{'x': 13.35012435913086, 'y': 5.78779840469360...","{'x': 0.00607309490442276, 'y': -0.00013958892...",...,"{'x': 0.0, 'y': 0.0, 'z': 0.0}","{'x': 0.0, 'y': 0.0, 'z': 0.0}",[],[],"[coffee maker, window blinds, thermostat, prin...","[{'unixTime': 1653696859605, 'systemTime': '17...",[],[],True,[]
8540,2,2449,17-14-21-89,1653696861895,False,"{'x': 0.00856788270175457, 'y': -0.00192505598...","{'x': 0.09976609796285629, 'y': -0.24180142581...","{'x': 0.1213906779885292, 'y': 0.0516977123916...","{'x': 13.992886543273926, 'y': 5.9014158248901...","{'x': 0.007854032330214977, 'y': -0.0011713962...",...,"{'x': 0.0, 'y': 0.0, 'z': 0.0}","{'x': 0.0, 'y': 0.0, 'z': 0.0}",[],[],"[coffee maker, window blinds, thermostat, prin...",[],[],[],True,[]


In [49]:
layerableOpenApps = framesDf.loc[(framesDf['IsLayerableMode'] == True)]
immersiveOpenApps = framesDf.loc[(framesDf['IsLayerableMode'] == False)]

immersiveOpenAppsDf = pandas.DataFrame(immersiveOpenApps["openApps"])
layerableOpenAppsDf = pandas.DataFrame(layerableOpenApps["openApps"])

immersiveOpenAppsDf.head(2)

for im in immersiveOpenAppsDf["openApps"]:
    if len(im) > 1:
        print(im)

avg = 0
count = 0
max = 0
min = 6
for lay in layerableOpenAppsDf["openApps"]:
    numAppOpen = len(lay)
    avg = numAppOpen + avg
    count = count + 1
    if(numAppOpen > max):
        max = numAppOpen
    if(numAppOpen < min):
        min = numAppOpen
    

print(avg/count)
print(max)
print(min)

3.517883247246857
5
0


In [50]:
gestureEvents
gestureEventsUnpacked = list(itertools.chain(*gestureEvents))
gestureEventsDf = pandas.DataFrame(gestureEventsUnpacked)

selectGesture = gestureEventsDf.loc[gestureEventsDf["action"] == "Select"]
manipulateGesture = gestureEventsDf.loc[gestureEventsDf["action"] == "Manipulate Action"]
navigateGesture = gestureEventsDf.loc[gestureEventsDf["action"] == "Navigation Action"]
holdGesture = gestureEventsDf.loc[gestureEventsDf["action"] == "Hold Action"]

In [51]:
leftRay = framesDf.loc[framesDf["leftHandRay"] != {'x': 0.0, 'y': 0.0, 'z': 0.0}]
rightRay = framesDf.loc[framesDf["rightHandRay"] != {'x': 0.0, 'y': 0.0, 'z': 0.0}]

In [52]:
#experimentEvents[0][0]
experimentEvents
pinCodes = experimentEvents[0][0]["pinCodes"]

for exper in experimentEvents:
    for exp in exper:
        if exp["eventName"] == "EndTask":
            print(exp)


{'unixTime': 1653696307255, 'systemTime': '17-05-07-25', 'userID': '1653694769600', 'sessionType': 'Mode B', 'eventName': 'EndTask', 'pinRandomSeed': 8831, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 1}
{'unixTime': 1653697293342, 'systemTime': '17-21-33-34', 'userID': '1653696859445', 'sessionType': 'Mode A', 'eventName': 'EndTask', 'pinRandomSeed': 9987, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 8}
{'unixTime': 1653697880692, 'systemTime': '17-31-20-69', 'userID': '1653697449084', 'sessionType': 'Mode B', 'eventName': 'EndTask', 'pinRandomSeed': 5967, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 6}
{'unixTime': 1653698364779, 'systemTime': '17-39-24-77', 'userID': '1653697449084', 'sessionType': 'Mode A', 'eventName': 'EndTask', 'pinRandomSeed': 5967, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 12}


In [53]:
experimentEvents
experimentsUnpacked = list(itertools.chain(*experimentEvents))

experimentsDf = pandas.DataFrame(experimentsUnpacked).sort_values(by="unixTime")
experimentsDf.tail(24)



,unixTime,systemTime,userID,sessionType,eventName,pinRandomSeed,pinNumCodes,pinCodeLength,pinNumPieces,pinCodes,score
0,1653695887249,16-58-07-24,1653694769600,Mode B,StartTask,8831,200,6,3,"[{'Value': 337773, 'Label': 'DH', 'Pieces': [{...",0
1,1653696307255,17-05-07-25,1653694769600,Mode B,EndTask,8831,200,6,3,[],1
2,1653696873385,17-14-33-38,1653696859445,Mode A,StartTask,9987,200,6,3,"[{'Value': 269070, 'Label': 'ZL', 'Pieces': [{...",0
3,1653697293342,17-21-33-34,1653696859445,Mode A,EndTask,9987,200,6,3,[],8
4,1653697460664,17-24-20-66,1653697449084,Mode B,StartTask,5967,200,6,3,"[{'Value': 134283, 'Label': 'SC', 'Pieces': [{...",0
5,1653697880692,17-31-20-69,1653697449084,Mode B,EndTask,5967,200,6,3,[],6
6,1653697944752,17-32-24-75,1653697449084,Mode A,StartTask,5967,200,6,3,"[{'Value': 134283, 'Label': 'SC', 'Pieces': [{...",0
7,1653698364779,17-39-24-77,1653697449084,Mode A,EndTask,5967,200,6,3,[],12


In [54]:
#check experiment events
immersiveStarts = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode B') & (experimentsDf['eventName'] == 'StartTask')]
immersiveEnds = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode B') & (experimentsDf['eventName'] == 'EndTask')]
immersiveStarts.reset_index(drop=True, inplace=True)
immersiveEnds.reset_index(drop=True, inplace=True)
immersiveEnds = immersiveEnds.assign(timeDifference = immersiveEnds['unixTime'].sub(immersiveStarts['unixTime'], fill_value=0))
immersiveTime = immersiveEnds['timeDifference'].sum() / 1000 / 60

layerableStarts = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode A') & (experimentsDf['eventName'] == 'StartTask')]
layerableEnds = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode A') & (experimentsDf['eventName'] == 'EndTask')]
layerableStarts.reset_index(drop=True, inplace=True)
layerableEnds.reset_index(drop=True, inplace=True)
layerableEnds = layerableEnds.assign(timeDifference = layerableEnds['unixTime'].sub(layerableStarts['unixTime'], fill_value=0))
layerableTime = layerableEnds['timeDifference'].sum() / 1000 / 60

experimentStarts = experimentsDf.loc[experimentsDf['eventName'] == 'StartTask']
experimentStops = experimentsDf.loc[experimentsDf['eventName'] == 'EndTask']

immersiveScore = experimentStops.loc[experimentStops['sessionType'] == 'Mode B']['score'].to_numpy()
layerableScore = experimentStops.loc[experimentStops['sessionType'] == 'Mode A']['score'].to_numpy()

In [55]:
immersiveStartTimes = experimentsDf.loc[(experimentsDf['sessionType'] == 'Mode B') & (experimentsDf['eventName'] == 'StartTask')]

In [56]:
codesEvents
codesUnpacked = list(itertools.chain(*codesEvents))

codesDf = pandas.DataFrame(codesUnpacked).sort_values(by="unixTime")
# codesDf.sort_values(by="unixTime")

In [57]:
successfulCodes = codesDf.loc[codesDf['success'] == True]
# successfulCodes.sort_values(by="unixTime")

In [58]:
appEvents
appsUnpacked = list(itertools.chain(*appEvents))
appsDf = pandas.DataFrame(appsUnpacked)
appsDf = appsDf.sort_values(by='eventTime')
#appsDf

In [59]:
#check time spent in each app

tutorialEvents = appsDf[appsDf['name'] == 'Tutorial']
annotationEvents = appsDf[appsDf['name'] == 'AnnotationsMenu']
codeEntryEvents = appsDf[appsDf['name'] == 'Code Entry']
itemInspectorEvents = appsDf[appsDf['name'] == 'Item Inspector']
deviceGroupsEvents = appsDf[appsDf['name'] == 'Device Groups']
atlasEvents = appsDf[appsDf['name'] == 'Atlas']

annotationStarts = annotationEvents[annotationEvents['activity'] == 'Start']
annotationStops = annotationEvents[annotationEvents['activity'] == 'Stop']
annotationStarts.reset_index(drop=True, inplace=True)
annotationStops.reset_index(drop=True, inplace=True)
annotationStops = annotationStops.assign(timeDifference = annotationStops['unixTime'].sub(annotationStarts['unixTime'], fill_value=0))
annotationTime = annotationStops['timeDifference'].sum() / 1000 / 60
print("time spent in annotation menu: " + str(annotationTime))

tutorialStarts = tutorialEvents[tutorialEvents['activity'] == 'Start']
tutorialStops = tutorialEvents[tutorialEvents['activity'] == 'Stop']
tutorialStops.reset_index(drop=True, inplace = True)
tutorialStarts.reset_index(drop = True, inplace = True)
tutorialStops = tutorialStops.assign(timeDifference = tutorialStops['unixTime'].sub(tutorialStarts['unixTime'],fill_value=0))
tutorialTime = tutorialStops['timeDifference'].sum() / 1000 / 60
print("time spent in tutorial: " + str(tutorialTime))

codeEntryStarts = codeEntryEvents[codeEntryEvents['activity'] == 'Start']
codeEntryStops = codeEntryEvents[codeEntryEvents['activity'] == 'Stop']
codeEntryStarts.reset_index(drop=True, inplace=True)
codeEntryStops.reset_index(drop=True, inplace=True)
codeEntryStops = codeEntryStops.assign(timeDifference = codeEntryStops['unixTime'].sub(codeEntryStarts['unixTime'], fill_value=0))
codeEntryTime = codeEntryStops['timeDifference'].sum() / 1000 / 60
print("time spent in code entry: " + str(codeEntryTime))

itemInspectorStarts = itemInspectorEvents[itemInspectorEvents['activity'] == 'Start']
itemInspectorStops = itemInspectorEvents[itemInspectorEvents['activity'] == 'Stop']
itemInspectorStarts.reset_index(drop=True, inplace=True)
itemInspectorStops.reset_index(drop=True, inplace=True)
itemInspectorStops = itemInspectorStops.assign(timeDifference = itemInspectorStops['unixTime'].sub(itemInspectorStarts['unixTime'], fill_value=0))
itemInspectorTime = itemInspectorStops['timeDifference'].sum() / 1000 / 60
print("time spent in item inspector: " + str(itemInspectorTime))

deviceGroupsStarts = deviceGroupsEvents[deviceGroupsEvents['activity'] == 'Start']
deviceGroupsStops = deviceGroupsEvents[deviceGroupsEvents['activity'] == 'Stop']
deviceGroupsStarts.reset_index(drop=True, inplace=True)
deviceGroupsStops.reset_index(drop=True, inplace=True)
deviceGroupsStops = deviceGroupsStops.assign(timeDifference = deviceGroupsStops['unixTime'].sub(deviceGroupsStarts['unixTime'], fill_value = 0))
deviceGroupsTime = deviceGroupsStops['timeDifference'].sum() / 1000 / 60
print("time spent in device groups: " + str(deviceGroupsTime))

atlasStarts = atlasEvents[atlasEvents['activity'] == 'Start']
atlasStops = atlasEvents[atlasEvents['activity'] == 'Stop']
atlasStarts.reset_index(drop=True, inplace=True)
atlasStops.reset_index(drop=True, inplace=True)
atlasStops = atlasStops.assign(timeDifference = atlasStops['unixTime'].sub(atlasStarts['unixTime'], fill_value=0))
atlasTime = atlasStops['timeDifference'].sum() / 1000 / 60
print("time spent in atlas: " + str(atlasTime))

print("\ntime spent in immersive mode: " + str(immersiveTime))
print("time spent in layerable mode: " + str(layerableTime))




time spent in annotation menu: 0.0
time spent in tutorial: 15.248866666666666
time spent in code entry: -19.06886666666667
time spent in item inspector: -23.013916666666667
time spent in device groups: -18.047483333333332
time spent in atlas: -17.2535

time spent in immersive mode: 14.000566666666666
time spent in layerable mode: 13.999733333333333


In [60]:

print("number of experiment starts: " + str(experimentStarts.shape[0]))
print("number of experiment stops: " + str(experimentStops.shape[0]))

number of experiment starts: 4
number of experiment stops: 4


In [61]:
print("number of correct codes entered: " + str(successfulCodes.shape[0]))
for score in immersiveScore:
    print("number of correct codes entered in immersive mode: " + str(score))
for score in layerableScore:
    print("number of correct codes entered in layerable mode: " + str(score))

number of correct codes entered: 28
number of correct codes entered in immersive mode: 1
number of correct codes entered in immersive mode: 6
number of correct codes entered in layerable mode: 8
number of correct codes entered in layerable mode: 12


In [62]:
placedObject = framesDf.loc[:,"placedObjects"]

for p in placedObject:
    if( len(p) < 15):
        print(len(p))

In [63]:
print("number of times left ray was tracked: " + str(leftRay.shape[0]))
print("number of times right ray was tracked: " + str(rightRay.shape[0]))

print("\nnumber of select gestures: " + str(selectGesture.shape[0]))
print("number of manipulate gestures: " + str(manipulateGesture.shape[0]))
print("number of navigate gestures: " + str(navigateGesture.shape[0]))
print("number of hold gestures: " + str(holdGesture.shape[0]))

number of times left ray was tracked: 2245
number of times right ray was tracked: 6838

number of select gestures: 20
number of manipulate gestures: 267
number of navigate gestures: 259
number of hold gestures: 4
